In [1]:
import os
import time
import csv
import re
import yaml
import inspect
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Move up a couple of folders
project_dir = os.path.abspath(os.path.join(current_dir, "../../"))

In [3]:
# Define the path to your YAML file
yaml_file = os.path.abspath(os.path.join(current_dir, "../../config.yaml"))

# Read the YAML file
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)

# Access the values from the YAML file
vessel_type = config['vessel_type']
class_type = config['class']
base = config['base']
quota = str(config['quota'])

In [4]:
#edit sesuai kebutuhan
num = base + 3
filenum = str(num)

In [5]:
# Directory file csv
read_path = os.path.join(project_dir,"input",vessel_type,quota,"split_{}.csv".format(filenum))
write_path = os.path.join(project_dir,"output",class_type, vessel_type,"{}.csv".format(filenum))

In [6]:
print(read_path)
print(write_path)

C:\Users\abinp\Documents\ITS\side projects\new\ship data scrapping\input\passanger\1000\split_3.csv
C:\Users\abinp\Documents\ITS\side projects\new\ship data scrapping\output\nk\passanger\3.csv


In [7]:
# Check if input file exists
if os.path.exists(read_path):
    print(f"CSV read file is found, continuing...")
else:
    print(f"CSV read file is not found")
    print('exiting....')
    
# Check if output file exists and create it if necessary
if os.path.exists(write_path):
    print(f"CSV write file  is found, continuing...")
else:
    print(f"CSV write file is not found, creating one...")
    os.makedirs(os.path.dirname(write_path), exist_ok=True) # Create output directory if it doesn't exist
    with open(write_path, 'w', newline='') as f:
        writer = csv.writer(f)

CSV read file is found, continuing...
CSV write file is not found, creating one...


In [8]:
def search(IMO):
    driver.get("https://www.classnk.com/register/regships/regships.aspx")

    search_field = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/table/tbody/tr[2]/td/input')))
    search_field.send_keys(IMO)

    search_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/p/a[1]')
    search_button.click()
    
    ship_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/div[1]/div[2]/h3/a')))
    ship_button.click()

In [9]:
def newtab():
    # Close all other windows except the original window
    original_window = driver.current_window_handle
    all_window_handles = driver.window_handles
    for window_handle in all_window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            driver.close()

    # Switch back to the original window
    driver.switch_to.window(original_window)

    # Open a new tab with the desired URL
    driver.execute_script("window.open('https://www.classnk.com/register/regships/one.aspx', '_blank');")

    # Switch to the new tab
    time.sleep(0.5)
    driver.switch_to.window(driver.window_handles[-1])

In [10]:
def find_xpath_by_text(driver, target_text):
    base_xpath = "/html/body/div/div/div[1]/div[3]/table/tbody/tr[NUMBER]"
    number = 1
    while True:
        xpath = base_xpath.replace("NUMBER", str(number))
        try:
            element = driver.find_element(By.XPATH, xpath)
            if target_text in element.text:
                return xpath
        except NoSuchElementException:
            # If the element is not found, stop the loop
            break
        number += 1
    
    # Return None if the target_text is not found
    return None

In [11]:
def shipdata():
    global LOA, LPP, B, H, T
    global DWT, NT, GT, LT
    global NAME, REGISTER, IMO, vessel_type
    global link
    
    wait = WebDriverWait(driver, 10) # Maximum wait time of 10 seconds
    
    target_text ="Ship's Name"
    result_xpath = find_xpath_by_text(driver,target_text)
    NAME = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    target_text ="Classification No."
    result_xpath = find_xpath_by_text(driver,target_text)
    REGISTER = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    target_text ="IMO No."
    result_xpath = find_xpath_by_text(driver,target_text)
    IMO = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    target_text ="Type of Ship -Purpose(intended service)"
    result_xpath = find_xpath_by_text(driver,target_text)
    vessel_type = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    
    
    #principal dimension
    target_text ="Overall Length (m)"
    result_xpath = find_xpath_by_text(driver,target_text)
    LOA = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    target_text ="Summer Draught (m)"
    result_xpath = find_xpath_by_text(driver,target_text)   
    T = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))

    target_text ="Registered LxBxD (m)"
    result_xpath = find_xpath_by_text(driver,target_text)
    dimensions = wait.until(EC.visibility_of_element_located((By.XPATH , '{}/td'.format(result_xpath))))
    overall = str(dimensions.text)    
    split = overall.split('x')
    LPP, B, H = split
    
    #ship tonnage
    target_text ="Tonnage Gross (Registered)"
    result_xpath = find_xpath_by_text(driver,target_text)
    GT = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))

    
    target_text ="Tonnage Net (Registered)"
    result_xpath = find_xpath_by_text(driver,target_text)
    NT = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    
    target_text ="Deadweight"
    result_xpath = find_xpath_by_text(driver,target_text)
    DWT = wait.until(EC.visibility_of_element_located((By.XPATH,'{}/td'.format(result_xpath))))
    
    link = 'https://www.vesselfinder.com/vessels/details/{}'.format(IMO.text)
    
    # print variable names and value
#     print('NAME:', NAME.text)
#     print('REGISTER:', REGISTER.text)
#     print('IMO:', IMO.text)
#     print('VESSEL TYPE:', vessel_type.text)
#     print('LOA:', LOA.text)
#     print('LPP:', LPP)
#     print('T:', T.text)
#     print('B:', B)
#     print('H:', H)
#     print('GT:', GT.text)
#     print('NT:', NT.text)
#     print('DWT:', DWT.text)
#     print('LINK:', link)

In [12]:
# Define the fieldnames for the CSV file
fieldnames = ['NAME', 'REGISTER', 'IMO', 'TYPE', 'LOA', 'LPP', 'B', 'H', 'T',  'GT', 'NT', 'DWT', 'Link']

In [13]:
# Set up the webdriver
driver = webdriver.Chrome()

# Counters for ships found and total IMOs
ship_count = 0
total_imo = 0

# Open the input file to get the total number of IMOs
with open(read_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    total_imo = sum(1 for _ in reader)

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()

    # Create a progress bar
    progress_bar = tqdm(total=total_imo, desc='Processing IMOs')

    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)

        for row in reader:
            IMO = row[0]
            progress_bar.update(1)
            progress_bar.set_postfix({'Ships Found ': ship_count})

            try:
                search(IMO)
                newtab()
                shipdata()
                ship_count += 1
                
            except:
                continue

            details_dict = {
                'NAME': NAME.text, 'REGISTER': REGISTER.text, 'IMO': IMO.text,
                'TYPE': vessel_type.text, 'LOA': LOA.text, 'LPP': LPP,
                'B': B, 'H': H, 'T': T.text, 'GT': GT.text,
                'NT': NT.text, 'DWT': DWT.text, 'Link': link
            }
            


    # Close the progress bar
    progress_bar.close()

# Close the webdriver
driver.quit()

# Display the total number of ships found
print('Total Ships Found:', ship_count)

Processing IMOs: 100%|███████████████████████████████████████████| 1001/1001 [1:13:53<00:00,  4.43s/it, Ships Found =0]


Total Ships Found: 0
